In [1]:
!pip install praw pandas


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import csv
import numpy as np
import pandas as pd
import praw
import prawcore
import random

from utils import *

In [6]:
import logging

# This will log everything
LOG = True
if LOG:
    handler = logging.StreamHandler()
    handler.setLevel(logging.DEBUG)
    for logger_name in ("praw", "prawcore"):
        logger = logging.getLogger(logger_name)
        logger.setLevel(logging.DEBUG)
        logger.addHandler(handler)

In [3]:
# Create the Reddit instance to do all the API stuff with
reddit = init_reddit()

In [7]:
keywords = [
    "trauma",
    "gaslight",
    "trigger",
    "toxic",
    "woke",
    "manipulat",
    "microaggression",
    "autis",
    "adhd",
    "ocd",
    "ptsd",
    "bipolar",
    "anxiety",
    "depress",
    "psycho",
    "delusion",
    "delulu",
    "solulu",
    "addict",
    "abus",
    "unstable"
]

# Searching for posts by keyword

This won't find comments with the keyword but whatever, could be useful for giving us an idea of which subreddits have which words

In [47]:
keyword_listings = search(
    reddit.subreddit("all"),
    " OR ".join(f"{keyword}~" for keyword in keywords),
)

In [59]:
with open("search-all-by-keyword.csv", "a") as file:
    writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)

    i = 0
    for submission in keyword_listings:
        writer.writerow(comment_relevant_fields(submission))
        i += 1
        if i > 10:
            break

# Getting comments from a post

In [3]:
listing = reddit.subreddit("unix").search('*', params={ "after": "t3_18zyt6g", "restrict_sr": True })
submissions = list(listing)
submissions

Fetching: GET https://oauth.reddit.com/r/unix/search/ at 1706734256.1732607
Data: None
Params: {'after': 't3_18zyt6g',
 'limit': 100,
 'q': '*',
 'raw_json': 1,
 'restrict_sr': True,
 'sort': 'relevance',
 'syntax': 'lucene',
 't': 'all'}
Response: 200 (62399 bytes) (rst-177:rem-599.0:used-1 ratelimit) at 1706734258.9487572


[Submission(id='1865xk2'),
 Submission(id='18qw8g9'),
 Submission(id='12pol3l'),
 Submission(id='17nn015'),
 Submission(id='18m7ea8'),
 Submission(id='17rre4b'),
 Submission(id='181hlcv'),
 Submission(id='zxulj4'),
 Submission(id='17jgifc'),
 Submission(id='18argdy'),
 Submission(id='18k6tli'),
 Submission(id='17tsw5v'),
 Submission(id='110nkii'),
 Submission(id='18jhs81'),
 Submission(id='17w07g9'),
 Submission(id='11bs899'),
 Submission(id='17l0qif'),
 Submission(id='17e2r50'),
 Submission(id='189vzyq'),
 Submission(id='17u0tl6'),
 Submission(id='18bcwv3'),
 Submission(id='17l8aq6'),
 Submission(id='17mtz7p'),
 Submission(id='17jpid4'),
 Submission(id='180l0ea'),
 Submission(id='16fc9dt'),
 Submission(id='184l9kb'),
 Submission(id='17lmjmt'),
 Submission(id='xpb2l8'),
 Submission(id='1856q95'),
 Submission(id='17vas0t'),
 Submission(id='10g0742'),
 Submission(id='17y11qr'),
 Submission(id='17tiiw2'),
 Submission(id='1816txd'),
 Submission(id='15swuun'),
 Submission(id='11wnw56'),
 Su

In [8]:
[s.url for s in submissions]

['https://www.reddit.com/r/unix/comments/1865xk2/some_help_with_openssl_fileenc_from_machine_a_to/',
 'https://www.idatum.net/upgrading-from-netbsd-51-to-10_rc1.html',
 'https://www.reddit.com/r/unix/comments/12pol3l/searching_for_the_most_unusual_desktop_environment/',
 'https://www.reddit.com/r/unix/comments/17nn015/beginner_question_how_to_properly_implement_dup2/',
 'https://www.delhi-magazine.com/education/overview-of-unix-operating-system/',
 'https://www.reddit.com/r/unix/comments/17rre4b/mongoose_not_accepting_callback/',
 'https://www.reddit.com/r/unix/comments/181hlcv/new_to_mac/',
 'https://www.reddit.com/r/unix/comments/zxulj4/fyi_grexorg_dies_on_2023_apr_15/',
 'https://www.reddit.com/r/unix/comments/17jgifc/kali_linux_error/',
 'https://byte-sized.de/uncategorized/freebsd-streaming-mit-chromium-moeglich-dank-widevine-support/#english',
 'https://freebsdfoundation.org/blog/2024-freebsd-community-survey-is-here/',
 'https://i.redd.it/jmkzvzez7zzb1.png',
 'https://i.redd.it/

In [9]:
comments = submissions[0].comments.list()
comments

In [11]:
comments[0].body

'Bash vs zsh won’t make a difference, but Sonoma has LibreSSL 3, and Mojave…doesn’t. I’m not sure what version it does have, but OpenSSL 1.0.2 was contemporary with Mojave so it cant be anything newer. I know for sure there were some breaking changes between 1.0.x and 1.1.x, and more between 1.1.x and 3.0.x.\n\nYou need to remember that OpenSSL is a suite of encryption primitives. Presentation will make a huge difference. The `openssl` command is really only intended as a developer/debugging tool for prototyping the library and not really meant to for end user facing purposes.\n\nTry these:\n\n* `openssl enc -e -aes-256-cbc -salt -a`\n* `openssl enc -d -aes-256-cbc -salt -a`\n\nThe `-a` uses ascii encoding and that may get around the issue you’re having. If not, then you’re probably completely out of luck with using openssl in this case.'

# Sampling comments randomly (by ID)

Since IDs are sequential, we can just sample them. But this won't guarantee all the keywords we want will show up

In [5]:
random.seed(0)

In [ ]:
# Run to restore state
with open("rand-state", "r") as file:
    random.setstate(eval(file.readline())) # todo deserialize properly

In [44]:
first_id = int("1000000", 36)  # todo find the ID of the first post we want to analyze
last_id = int("k000000", 36)  # todo find the ID of the last post we want to analyze
with open("sample-all.csv", "a") as file:
    writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
    while True:
        # IDs must be in lowercase base 36
        ids = [
            "t1_" + np.base_repr(random.randrange(first_id, last_id), 36).lower()
            for _ in range(100)
        ]
        # Ask for info on 100 IDs at once because PRAW makes batches of 100
        info = reddit.info(fullnames=ids)
        try:
            for submission in info:
                # Only keep comments
                if (
                    submission.name.startswith("t1")
                    and submission.body != "[removed]"
                    and submission.body != "[deleted]"
                ):
                    writer.writerow(comment_relevant_fields(submission))
        except Exception as e:
            print(e, e.__class__, id)
            break

KeyboardInterrupt: 

In [45]:
# Run to save state to pick back up later
with open("rand-state.txt", "w") as file:
    file.write(str(random.getstate())) # todo serialize properly

## Load comments

In [8]:
COMMENT_COLS = ["name", "subreddit", "body"]

df = pd.read_csv("sample-all.csv", names=COMMENT_COLS)
df.drop_duplicates(inplace=True)
df["body"] = df["body"].map(str)
df

,name,subreddit,body
0,t1_ei5yh1p,Granblue_en,Thank you! So it was working as intended. And ...
1,t1_ftj6bfm,worldnews,I am having a productive discussion by calling...
2,t1_ikv838q,baseballcards,I bought a box of NFL cards at a card show las...
3,t1_c7ql4x6,gonewildcurvy,How could I gorget that gorgeous body and that...
4,t1_hps8193,dankmemes,downvote this comment if the meme sucks. upvot...
...,...,...,...
33581,t1_d3ehsro,pcmasterrace,1) Nope 2) A lot of games support this as lon...
33582,t1_cuejaxj,SquaredCircle,WTF
33583,t1_ikwnt5c,horizon,Gameplay ideas: 1. Repeating missions- like s...
33584,t1_cuca0wi,funny,No


In [9]:
def matches(body):
    for keyword in keywords:
        if keyword in body:
            return True
    return False

In [10]:
df["body"] = df["body"].map(lambda body: body.lower().strip())
matched_rows = df[df["body"].map(matches)]
matched_rows

,name,subreddit,body
22,t1_fszn69a,Showerthoughts,i think it’s actually the massive amounts of m...
55,t1_ei0m7ng,Granblue_en,"yeah, i actually really like how they didn't j..."
109,t1_i1bmnrr,u/jesterboyd,one of the problems is that people on the far-...
191,t1_icu1wl8,Damnthatsinteresting,yes new york is so beautiful so many homeless ...
330,t1_g139g81,AmItheAsshole,yta. shaming people for their weight doesn’t w...
...,...,...,...
32966,t1_ch46hhx,MMA,there was quote from the smashing machine hbo ...
33010,t1_j3jrck4,parentsnark,yesssss! i recently unfollowed sterna because ...
33018,t1_hzpcffu,LockUppOTT,munawar telling payal there’s no way teams are...
33330,t1_cl83csn,ptsd,it hasn't been since i was a kid. it's not jus...


In [12]:
{
    keyword: len(matched_rows[matched_rows["body"].map(lambda body: keyword in body)])
    for keyword in keywords
}

{'trauma': 30,
 'gaslight': 6,
 'trigger': 64,
 'toxic': 33,
 'woke': 26,
 'manipulat': 37,
 'microaggression': 0,
 'autis': 14,
 'adhd': 11,
 'ocd': 4,
 'ptsd': 9,
 'bipolar': 3,
 'anxiety': 44,
 'depress': 59,
 'psycho': 42,
 'delusion': 18,
 'delulu': 0,
 'solulu': 0,
 'addict': 59,
 'abus': 91,
 'unstable': 9}